### Set styling for plotting

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns
sns.set_palette('colorblind')
from matplotlib.pyplot import tight_layout
# ##SETTING PARAMS FOR MATPLOTLIB FIGURES
plt.rcParams.update({"figure.figsize": (6, 6),
                 "axes.facecolor": "white",
                 "axes.edgecolor": "black"})
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=sns.color_palette('colorblind'))
##set font size
font = {'family': 'sans-serif',
       'weight': 'normal',
       'size': 14}
plt.rc('font', **font)
# ##PANDAS PLOTTING
pd.plotting.register_matplotlib_converters()

### Step 1: save environment file

In [2]:
!conda env export > nas_environment.yml

### Step 2: import modules

In [3]:
import glob
import os
import xml.etree.ElementTree as ET
from datetime import datetime, timezone
from tqdm import tqdm
import codecs
import csv
import numpy as np
!pip install openpyxl
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


### Step 3: import data files

In [4]:
data_directory_xl = "/home/paulharford/college/project/project_data/nas/WEATHERED Report CAD 20241217.xlsx"
full_path_xl = os.path.abspath(data_directory_xl)

In [5]:
##Import data from excel into panda's dataframe
nas_df = pd.read_excel(full_path_xl)

In [6]:
nas_df.head(10)

,Randomised ID,DateCallForPerf,Time of Day,PickupTown,EIRCODE ID,DespatchCode,DespCodeDescription,Hospital Attended
0,54959224086,2016-01-01,Night,DUNGLOE,NaN,17B01P,Falls - Possibly Dangerous body Area - Public ...,LETTERKENNY GENERAL HOSPITAL
1,54959224146,2016-01-01,Night,BEAUMONT DUBLIN 9,NaN,17B01G,Falls - Possibly Dangerous body Area - On the ...,BEAUMONT HOSPITAL
2,54959224172,2016-01-01,Night,DROGHEDA,NaN,17B01,Falls - Possibly Dangerous body Area -,LOURDES HOSPITAL DROGHEDA
3,54959224182,2016-01-01,Night,BORRISOKANE,NaN,17B01P,Falls - Possibly Dangerous body Area - Public ...,PORTIUNCULA GENERAL HOSPITAL
4,54959224201,2016-01-01,Night,WEXFORD,NaN,17B01G,Falls - Possibly Dangerous body Area - On the ...,NaN
5,54959224289,2016-01-01,Night,KILLARNEY,NaN,17D04G,Falls - Not Alert - On the ground or floor,KERRY UNIVERSITY HOSPITAL
6,54959224429,2016-01-01,Night,RATHNEENY WEST LAGHEY,NaN,17B01,Falls - Possibly Dangerous body Area -,SLIGO GENERAL HOSPITAL
7,54959224658,2016-01-01,Morning,GLENAGEARY,NaN,17B00G,Falls - Bravo Override - On the ground or floor,ST VINCENTS HOSPITAL
8,54959224681,2016-01-01,Morning,DUNDALK,NaN,17B01,Falls - Possibly Dangerous body Area -,LOURDES HOSPITAL DROGHEDA
9,54959224682,2016-01-01,Morning,CAUSEWAY,NaN,17B01G,Falls - Possibly Dangerous body Area - On the ...,KERRY UNIVERSITY HOSPITAL


### Step 4.1: Check Hospital Attended colund for list of hospitals for mapping to regions

In [7]:
unique_hospitals = nas_df["Hospital Attended"].unique()
print(unique_hospitals)


['LETTERKENNY GENERAL HOSPITAL' 'BEAUMONT HOSPITAL'
 'LOURDES HOSPITAL DROGHEDA' 'PORTIUNCULA GENERAL HOSPITAL' nan
 'KERRY UNIVERSITY HOSPITAL' 'SLIGO GENERAL HOSPITAL'
 'ST VINCENTS HOSPITAL' 'UNIVERSITY HOSPITAL LIMERICK'
 'SOUTH TIPPERARY GENERAL' 'MULLINGAR REGIONAL HOSPITAL'
 'NAAS GENERAL HOSPITAL' 'CORK UNIVERSITY HOSPITAL'
 'PORTLAOISE GENERAL HOSPITAL' 'CONNOLLY MEMORIAL HOSPITAL'
 'CAVAN GENERAL HOSPITAL' 'MAYO GENERAL HOSPITAL'
 'UNIVERSITY HOSPITAL GALWAY' 'WATERFORD UNIVERSITY HOSPITAL'
 'TULLAMORE GENERAL HOSPITAL' 'WEXFORD GENERAL HOSPITAL'
 'ST JAMES HOSPITAL' 'MERCY UNIVERSITY HOSPITAL'
 'ST MICHAELS DUN LAOGHAIRE' 'OUR LADYS NAVAN' 'TALLAGHT HOSPITAL'
 'MATER HOSPITAL DUBLIN' 'BANTRY GENERAL HOSPITAL'
 'ST LUKES HOSPITAL KILKENNY' 'SACRED HEART HOSPITAL MAYO'
 'ST COLUMCILLES HOSPITAL LOUGHLINSTOWN' 'MALLOW GENERAL HOSPITAL'
 'DAISY HILL HOSPTIAL NEWRY' 'ALTNAGELVIN HOSPITAL'
 'CORK UNIVERSITY MATERNITY HOSP' 'BON SECOURS HOSPITAL CORK'
 'ENNIS GENERAL HOSPITAL' 'ROS

In [8]:
##add regional information
hse_mapping = {
    "HSE Dublin and North East": [
        "BEAUMONT HOSPITAL", "LOURDES HOSPITAL DROGHEDA", "CAVAN GENERAL HOSPITAL",
        "OUR LADYS NAVAN", "MATER HOSPITAL DUBLIN", "CONNOLLY MEMORIAL HOSPITAL",
        "LOUTH COUNTY HOSPITAL"
    ],
    "HSE Dublin and Midlands": [
        "MULLINGAR REGIONAL HOSPITAL", "NAAS GENERAL HOSPITAL",
        "PORTLAOISE GENERAL HOSPITAL", "TULLAMORE GENERAL HOSPITAL",
        "ST JAMES HOSPITAL", "TALLAGHT HOSPITAL"
    ],
    "HSE Dublin and South East": [
        "ST VINCENTS HOSPITAL", "ST MICHAELS DUN LAOGHAIRE", "HOLLES STREET MATERNITY HOSPITAL",
        "ST COLUMCILLES HOSPITAL LOUGHLINSTOWN", "ST LUKES HOSPITAL KILKENNY",
        "SOUTH TIPPERARY GENERAL HOSPITAL", "WATERFORD UNIVERSITY HOSPITAL",
        "WEXFORD GENERAL HOSPITAL","SOUTH TIPPERARY GENERAL"
    ],
    "HSE South East": [
        "CORK UNIVERSITY HOSPITAL", "CORK UNIVERSITY MATERNITY HOSP",
        "MERCY UNIVERSITY HOSPITAL", "MALLOW GENERAL HOSPITAL", "BANTRY GENERAL HOSPITAL",
        "SOUTH INFIRMARY-VICTORIA UNIVERSITY HOSPITAL", "ST FINBARRS HOSPITAL CORK", "KERRY UNIVERSITY HOSPITAL"
    ],
    "HSE Mid West": [
        "UNIVERSITY HOSPITAL LIMERICK", "ENNIS GENERAL HOSPITAL", "NENAGH GENERAL HOSPITAL",
        "ST JOHNS HOSPITAL LIMERICK", "LIMERICK MATERNITY HOSPITAL"
    ],
    "HSE West and North West": [
        "LETTERKENNY GENERAL HOSPITAL", "PORTIUNCULA GENERAL HOSPITAL",
        "MAYO GENERAL HOSPITAL", "UNIVERSITY HOSPITAL GALWAY", "ROSCOMMON GENERAL HOSPITAL",
        "SLIGO GENERAL HOSPITAL", "MERLIN PARK HOSPITAL"
    ]
}



### Step 4.2: Map hospitals to regions 

In [9]:
def categorize_hospital(hospital_name):
    # Check if hospital_name is a valid string, otherwise return "Hospital_Unknown"
    if not isinstance(hospital_name, str):
        return "Unknown"
    
    for region, hospitals in hse_mapping.items():
        if hospital_name.upper() in hospitals:
            return region
    return "Hospital Unknown"

In [10]:
nas_df["region"] = nas_df["Hospital Attended"].apply(categorize_hospital)

In [11]:
nas_df.head(15)

,Randomised ID,DateCallForPerf,Time of Day,PickupTown,EIRCODE ID,DespatchCode,DespCodeDescription,Hospital Attended,region
0,54959224086,2016-01-01,Night,DUNGLOE,NaN,17B01P,Falls - Possibly Dangerous body Area - Public ...,LETTERKENNY GENERAL HOSPITAL,HSE West and North West
1,54959224146,2016-01-01,Night,BEAUMONT DUBLIN 9,NaN,17B01G,Falls - Possibly Dangerous body Area - On the ...,BEAUMONT HOSPITAL,HSE Dublin and North East
2,54959224172,2016-01-01,Night,DROGHEDA,NaN,17B01,Falls - Possibly Dangerous body Area -,LOURDES HOSPITAL DROGHEDA,HSE Dublin and North East
3,54959224182,2016-01-01,Night,BORRISOKANE,NaN,17B01P,Falls - Possibly Dangerous body Area - Public ...,PORTIUNCULA GENERAL HOSPITAL,HSE West and North West
4,54959224201,2016-01-01,Night,WEXFORD,NaN,17B01G,Falls - Possibly Dangerous body Area - On the ...,NaN,Unknown
5,54959224289,2016-01-01,Night,KILLARNEY,NaN,17D04G,Falls - Not Alert - On the ground or floor,KERRY UNIVERSITY HOSPITAL,HSE South East
6,54959224429,2016-01-01,Night,RATHNEENY WEST LAGHEY,NaN,17B01,Falls - Possibly Dangerous body Area -,SLIGO GENERAL HOSPITAL,HSE West and North West
7,54959224658,2016-01-01,Morning,GLENAGEARY,NaN,17B00G,Falls - Bravo Override - On the ground or floor,ST VINCENTS HOSPITAL,HSE Dublin and South East
8,54959224681,2016-01-01,Morning,DUNDALK,NaN,17B01,Falls - Possibly Dangerous body Area -,LOURDES HOSPITAL DROGHEDA,HSE Dublin and North East
9,54959224682,2016-01-01,Morning,CAUSEWAY,NaN,17B01G,Falls - Possibly Dangerous body Area - On the ...,KERRY UNIVERSITY HOSPITAL,HSE South East


In [12]:
##check for NaN/null values
(nas_df.isna().sum()/nas_df.shape[0] * 100).sort_values()

Randomised ID           0.000000
DateCallForPerf         0.000000
Time of Day             0.000000
DespatchCode            0.000000
DespCodeDescription     0.000000
region                  0.000000
PickupTown              0.008376
EIRCODE ID              8.379831
Hospital Attended      15.922065
dtype: float64

In [13]:
##for the hospital attended that are nana if we have the pickup town we can infer the Hospital attended
## this is not always teh case as patients can be brought to hospitals outsode of their region
## but as 15% of the data is missing its reasonable to assume that the majoring of patiemst went to theire local hospital
unknown_hospitals_df = nas_df[nas_df["region"] == "Unknown"][["Hospital Attended", "PickupTown"]].drop_duplicates()
unknown_hospitals_df.head()



,Hospital Attended,PickupTown
4,NaN,WEXFORD
16,NaN,CARROWCUBIC BALLYCASTLE
23,NaN,SHANKILL DUBLIN 18
30,NaN,CELBRIDGE
46,NaN,NAVAN


In [14]:
unique_pickup_towns = unknown_hospitals_df["PickupTown"].unique()

# Print the unique PickupTown values
print(unique_pickup_towns)

['WEXFORD' 'CARROWCUBIC BALLYCASTLE' 'SHANKILL DUBLIN 18' ...
 'MASINASS CREESLOUGH' 'CARRIGLEA DUNGARVAN' 'KILLAREE CHARLEVILLE']


In [15]:
nas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155206 entries, 0 to 155205
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   Randomised ID        155206 non-null  int64         
 1   DateCallForPerf      155206 non-null  datetime64[ns]
 2   Time of Day          155206 non-null  object        
 3   PickupTown           155193 non-null  object        
 4   EIRCODE ID           142200 non-null  object        
 5   DespatchCode         155206 non-null  object        
 6   DespCodeDescription  155206 non-null  object        
 7   Hospital Attended    130494 non-null  object        
 8   region               155206 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 10.7+ MB


### Step 4.3: For Hospital Attended = unknown use the pickup town to determine hospital 

In [90]:
town_to_county = {
    "achill": "Mayo",
    "adare": "Limerick",
    "aghada": "Cork",
    "ahascragh": "Galway",
    "aherla": "Cork",
    "aherlow": "Tipperary",
    "aille": "Galway",
    "aillroe": "Clare",
    "ailt an Chorráin": "Donegal",
    "aithnín": "Galway",
    "allen": "Kildare",
    "allenwood": "Kildare",
    "allihies": "Cork",
    "an Cheathrú Rua": "Galway",
    "an Clochán Liath": "Donegal",
    "an Fál Carrach": "Donegal",
    "an Móta": "Galway",
    "an Spidéal": "Galway",
    "an Uaimh": "Meath",
    "anascaul": "Kerry",
    "annacotty": "Limerick",
    "annagassan": "Louth",
    "annagry": "Donegal",
    "annascaul": "Kerry",
    "ardagh": "Longford",
    "ardara": "Donegal",
    "ardee": "Louth",
    "ardfert": "Kerry",
    "ardfinnan": "Tipperary",
    "ardmore": "Waterford",
    "ardrahan": "Galway",
    "ardrossan": "Down",
    "arklow": "Wicklow",
    "arless": "Laois",
    "armagh": "Armagh",
    "arva": "Cavan",
    "askeaton": "Limerick",
    "athenry": "Galway",
    "athlone": "Westmeath",
    "athleague": "Roscommon",
    "athlacca": "Limerick",
    "attymon": "Galway",
    "aughrim": "Galway",
    "aughnacliffe": "Longford",
    "aughnacloy": "Tyrone",
    "aughrim": "Wicklow",
    "avoca": "Wicklow",
    "bagenalstown": "Carlow",
    "bailieborough": "Cavan",
    "balbriggan": "Dublin",
    "baldoyle": "Dublin",
    "balla": "Mayo",
    "ballaghaderreen": "Roscommon",
    "ballina": "Mayo",
    "ballinagh": "Cavan",
    "ballinakill": "Laois",
    "ballinalee": "Longford",
    "ballinamore": "Leitrim",
    "ballinasloe": "Galway",
    "ballincollig": "Cork",
    "ballindine": "Mayo",
    "ballineen": "Cork",
    "ballingarry": "Limerick",
    "ballinhassig": "Cork",
    "ballinlough": "Roscommon",
    "ballinrobe": "Mayo",
    "ballinspittle": "Cork",
    "ballinteer": "Dublin",
    "ballintra": "Donegal",
    "ballintubber": "Mayo",
    "ballisodare": "Sligo",
    "ballivor": "Meath",
    "ballybay": "Monaghan",
    "ballybofey": "Donegal",
    "ballyboughal": "Dublin",
    "ballycastle": "Mayo",
    "ballyconnell": "Cavan",
    "ballycotton": "Cork",
    "ballycroy": "Mayo",
    "ballydehob": "Cork",
    "ballydesmond": "Cork",
    "ballyduff": "Waterford",
    "ballyfarnon": "Roscommon",
    "ballyferriter": "Kerry",
    "ballyfoyle": "Kilkenny",
    "ballygar": "Galway",
    "ballygarrett": "Wexford",
    "ballyhaise": "Cavan",
    "ballyhale": "Kilkenny",
    "ballyhaunis": "Mayo",
    "ballyheigue": "Kerry",
    "ballyjamesduff": "Cavan",
    "ballylanders": "Limerick",
    "ballyleague": "Roscommon",
    "ballylinan": "Laois",
    "ballylongford": "Kerry",
    "ballylynan": "Laois",
    "ballymahon": "Longford",
    "ballymote": "Sligo",
    "ballymun": "Dublin",
    "ballynahinch": "Down",
    "ballynacally": "Clare",
    "ballynacargy": "Westmeath",
    "ballynahinch": "Galway",
    "ballyragget": "Kilkenny",
    "ballyroan": "Laois",
    "ballysadare": "Sligo",
    "ballyshannon": "Donegal",
    "ballyvaughan": "Clare",
    "ballywalter": "Down",
    "balrothery": "Dublin",
    "baltinglass": "Wicklow",
    "bandon": "Cork",
    "bantry": "Cork",
    "bawnboy": "Cavan",
    "bearna": "Galway",
    "beaufort": "Kerry",
    "bective": "Meath",
    "belcoo": "Fermanagh",
    "belfast": "Antrim",
    "bellaghy": "Londonderry",
    "bellahy": "Sligo",
    "bellavary": "Mayo",
    "belleek": "Fermanagh",
    "belmullet": "Mayo",
    "belturbet": "Cavan",
    "bennettsbridge": "Kilkenny",
    "berrings": "Cork",
    "bettystown": "Meath",
    "bishopstown": "Cork",
    "blacklion": "Cavan",
    "blackrock": "Dublin",
    "blackwater": "Wexford",
    "blessington": "Wicklow",
    "blarney": "Cork",
    "blennerville": "Kerry",
    "boherbue": "Cork",
    "borris": "Carlow",
    "borris-in-Ossory": "Laois",
    "borrisokane": "Tipperary",
    "borrisoleigh": "Tipperary",
    "bray": "Wicklow",
    "bruree": "Limerick",
    "bunclody": "Wexford",
    "bundoran": "Donegal",
    "bunmahon": "Waterford",
    "bunratty": "Clare",
    "buttevant": "Cork",
    "cabinteely": "Dublin",
    "cahersiveen": "Kerry",
    "cahir": "Tipperary",
    "cahirciveen": "Kerry",
    "cappamore": "Limerick",
    "cappawhite": "Tipperary",
    "carbury": "Kildare",
    "carlingford": "Louth",
    "carlow": "Carlow",
    "carraroe": "Galway",
    "carndonagh": "Donegal",
    "carrick-on-Shannon": "Leitrim",
    "carrick-on-Suir": "Tipperary",
    "carrickmacross": "Monaghan",
    "carrigaline": "Cork",
    "carrigallen": "Leitrim",
    "carrigtwohill": "Cork",
    "cashel": "Tipperary",
    "castlebar": "Mayo",
    "castleblayney": "Monaghan",
    "castlecomer": "Kilkenny",
    "castleconnell": "Limerick",
    "castledermot": "Kildare",
    "castleisland": "Kerry",
    "castleknock": "Dublin",
    "castlemartyr": "Cork",
    "castlerea": "Roscommon",
    "castletownbere": "Cork",
    "castletownshend": "Cork",
    "celbridge": "Kildare",
    "clane": "Kildare",
    "clara": "Offaly",
    "claremorris": "Mayo",
    "clifden": "Galway",
    "clogherhead": "Louth",
    "clonakilty": "Cork",
    "clondalkin": "Dublin",
    "clonegal": "Carlow",
    "clones": "Monaghan",
    "clonmel": "Tipperary",
    "clonroche": "Wexford",
    "coachford": "Cork",
    "collon": "Louth",
    "cong": "Mayo",
    "convoy": "Donegal",
    "cootehill": "Cavan",
    "cork": "Cork",
    "courtown": "Wexford",
    "crumlin": "Dublin",
    "dalkey": "Dublin",
    "dingle": "Kerry",
    "doolin": "Clare",
    "drogheda": "Louth",
    "dromahair": "Leitrim",
    "dromore": "Down",
    "dublin": "Dublin",
    "dun Laoghaire": "Dublin",
    "dundalk": "Louth",
    "dunfanaghy": "Donegal",
    "dungarvan": "Waterford",
    "dungloe": "Donegal",
    "dunleer": "Louth",
    "dunmanway": "Cork",
    "edenderry": "Offaly",
    "enfield": "Meath",
    "ennis": "Clare",
    "enniscorthy": "Wexford",
    "ennistymon": "Clare",
    "fermoy": "Cork",
    "fethard": "Tipperary",
    "foxford": "Mayo",
    "galway": "Galway",
    "glenties": "Donegal",
    "gorey": "Wexford",
    "gort": "Galway",
    "graiguenamanagh": "Kilkenny",
    "granard": "Longford",
    "greystones": "Wicklow",
    "headford": "Galway",
    "holywood": "Down",
    "kilbeggan": "Westmeath",
    "kilcock": "Kildare",
    "kilcoole": "Wicklow",
    "kilcullen": "Kildare",
    "kildare": "Kildare",
    "kilkee": "Clare",
    "kilkenny": "Kilkenny",
    "killarney": "Kerry",
    "killorglin": "Kerry",
    "kilrush": "Clare",
    "kiltimagh": "Mayo",
    "kinnegad": "Westmeath",
    "laois": "Laois",
    "leixlip": "Kildare",
    "letterkenny": "Donegal",
    "lifford": "Donegal",
    "limerick": "Limerick",
    "lisburn": "Antrim",
    "lisdoonvarna": "Clare",
    "longford": "Longford",
    "loughrea": "Galway",
    "lucan": "Dublin",
    "malahide": "Dublin",
    "mallow": "Cork",
    "maynooth": "Kildare",
    "midleton": "Cork",
    "mitchelstown": "Cork",
    "monaghan": "Monaghan",
    "mullingar": "Westmeath",
    "naas": "Kildare",
    "navan": "Meath",
    "nenagh": "Tipperary",
    "newbridge": "Kildare",
    "newcastle West": "Limerick",
    "newmarket-on-Fergus": "Clare",
    "newport": "Mayo",
    "newry": "Down",
    "newtownards": "Down",
    "oughterard": "Galway",
    "portarlington": "Laois",
    "portlaoise": "Laois",
    "rathangan": "Kildare",
    "rathdowney": "Laois",
    "roscommon": "Roscommon",
    "rush": "Dublin",
    "sallins": "Kildare",
    "shannon": "Clare",
    "skibbereen": "Cork",
    "sligo": "Sligo",
    "strabane": "Tyrone",
    "terenure": "Dublin",
    "thurles": "Tipperary",
    "tipperary": "Tipperary",
    "trim": "Meath",
    "tuam": "Galway",
    "tullamore": "Offaly",
    "tullow": "Carlow",
    "waterford": "Waterford",
    "westport": "Mayo",
    "wexford": "Wexford",
    "wicklow": "Wicklow",
    "youghal": "Cork",
    "glenageary": "Dublin",
    "tyrrellspass": "Westmeath",
    "donegal town": "Donegal",
    "derryvarroge donadea": "Kildare",
    "kerry pike carrigrohane": "Cork",
    "milltown": "Mayo",
    "mullaghmore north tedavnet": "Monaghan",
    "tralee": "Kerry",
    "grangehiggin kilmeague": "Kildare",
    "caherciveen": "Kerry",
    "ratoath": "Meath",
    "reenard south caherciveen": "Kerry",
    "ballymullen tralee": "Kerry",
    "crosspatrick": "Kilkenny",
    "multyfarnham": "Westmeath",
    "tonroe oranmore": "Galway",
    "cavan": "Cavan",
    "4 park business centre farranfore": "Kerry",
    "abberley killiney": "Dublin",
    "abbeyderg keenagh": "Longford",
    "abbeydorney": "Kerry",
    "abbeyfeale": "Limerick",
    "abbeyknockmoy": "Galway",
    "abbeyland eyrecourt": "Galway",
    "abbeylara": "Longford",
    "abbeyleix": "Laois",
    "abbeymahon timoleague": "Cork",
    "abbeytown boyle": "Roscommon",
    "achonry": "Sligo",
    "aclamon new ross": "Wexford",
    "aclare": "Sligo",
    "aclare myshall": "Carlow",
    "acontha durrow": "Laois",
    "acragar mountmellick": "Laois",
    "acres knockatallon": "Monaghan",
    "adamstown": "Wexford",
    "adamstown castletown geoghegan": "Westmeath",
    "adderwal doochary": "Donegal",
    "addinstown delvin": "Westmeath",
    "admiran stranorlar": "Donegal",
    "adoon gorvagh": "Leitrim",
    "adrigole": "Cork",
    "agall screggan": "Offaly",
    "aggard craughwell": "Galway",
    "aghabeg corries muine bheag / ": "Carlow",
    "aghabeg east lixnaw": "Kerry",
    "aghabeg middle lixnaw": "Kerry",
    "aghabeg west lixnaw": "Kerry",
    "aghaboy killoe": "Longford",
    "aghaboy north tydavnet": "Monaghan",
    "aghacarra corrigeenroe boyle": "Roscommon",
    "aghacarrible lispole": "Kerry",
    "aghaclay smithborough": "Monaghan",
    "aghacloghger strokestown": "Roscommon",
    "aghacoora lixnaw": "Kerry",
    "aghade kilbride": "Carlow",
    "aghadiffin kilkelly": "Mayo",
    "aghadoe killeagh": "Cork",
    "aghadoe woods barleymount kill": "Kerry",
    "aghadouglas brosna birr": "Offaly",
    "aghadreen wolfhill": "Laois",
    "aghadreenagh redhills": "Cavan",
    "aghadrumkeen newbliss": "Monaghan",
    "aghadrumdoney scotshouse": "Monaghan",
    "aghafin edgeworthstown": "Longford",
    "aghagad grange": "Sligo",
    "aghagah moyne": "Longford",
    "aghagegna crosserlough": "Cavan",
    "aghagoldrick stradone": "Cavan",
    "aghagower castlecoote": "Roscommon",
    "aghakilbrack drumshanbo": "Leitrim",
    "aghakilfaughna mohill": "Leitrim",
    "aghalateeve glenade": "Leitrim",
    "aghaloora kilnaleck": "Cavan",
    "aghalough aghabog": "Monaghan",
    "aghamackalinn carrickroe": "Monaghan",
    "aghamannan kilglass": "Roscommon",
    "aghameelik clonbullogue": "Offaly",
    "aghamore causeway": "Kerry",
    "aghamore hillstreet carrick-on": "Leitrim",
    "aghamore lower moyne": "Longford",
    "aghamuck kilrooskey": "Roscommon",
    "aghanageeragh edgeworthstown": "Longford",
    "aghanagran middle ballylongfor": "Kerry",
    "aghanargit moate": "Westmeath",
    "aghanlish kinlough": "Leitrim",
    "aghareagh west scotshouse": "Monaghan",
    "agharrow ballinfull": "Sligo",
    "aghatubrid leap": "Cork",
    "aghavadrin killeshandra": "Cavan",
    "aghavanny kiltyclogher": "Leitrim",
    "aghavas": "Leitrim",
    "aghaveans loch gowna": "Cavan",
    "aghawee crosserlough": "Cavan",
    "agher summerhill": "Meath",
    "agherakeltan scotstown": "Monaghan",
    "aghmagree kilteevan": "Roscommon",
    "aghnacor killeshandra": "Cavan",
    "aghnacreevy killeshandra": "Cavan",
    "aghnagap castleshane": "Monaghan",
    "aghnagore cloondara": "Longford",
    "aghnahaha rossinver": "Leitrim",
    "aghnahola scotshouse": "Monaghan",
    "aghnahunshin roosky": "Roscommon",
    "aghnasillagh carrickboy": "Longford",
    "aghnaskeagh mountnugent": "Cavan",
    "aghnaskew scotshouse": "Monaghan",
    "aghnasurn corrigeenroe boyle": "Roscommon",
    "aglish cappoquin": "Waterford",
    "aglish lispole": "Kerry",
    "aglish roscrea": "Tipperary",
    "ahafona ballybunnion": "Kerry",
    "ahalana moyvane": "Kerry",
    "ahane knocknagoshel": "Kerry",
    "aharas ballingeary": "Cork",
    "ahare castletown": "Wexford",
    "ahima ballybunnion": "Kerry",
    "aighan bruckless": "Donegal",
    "aillwee rosmuck": "Galway",
    "airmount slieverue": "Kilkenny",
    "alasty kill": "Kildare",
    "alderbrook ashbourne": "Meath",
    "altachullion upper swanlinbar": "Cavan",
    "altagowlan arigna carrick-on-s": "Leitrim",
    "altahalla clonmany": "Donegal",
    "altans templeboy": "Sligo",
    "altavilla coolrain": "Laois",
    "altmush kilmainhamwood": "Meath",
    "ambrosetown duncormick": "Wexford",
    "amorset manorhamilton": "Leitrim",
    "an grianan termonfeckin": "Louth",
    "anaverna ravensdale": "Louth",
    "anglesboro kilmallock": "Limerick",
    "ankerland castlepollard": "Westmeath",
    "annacarty": "Tipperary",
    "annaduff glebe annaduff carric": "Leitrim",
    "annafarney shercock": "Cavan",
    "annagh birr": "Offaly",
    "annagh clonmany": "Donegal",
    "annagh killenagh": "Wexford",
    "annagh moyne": "Longford",
    "annagh oldcastle": "Meath",
    "annagh the pigeons": "Cavan",
    "annagharnet virginia": "Cavan",
    "annaghasna kilnagross carrick-": "Leitrim",
    "annaghbeg gneeveguilla": "Kerry",
    "annaghbeg strokestown": "Roscommon",
    "annaghbrack blue ball": "Offaly",
    "annaghcarthy drumnacool": "Leitrim",
    "annaghcooleen bornacoola": "Leitrim",
    "annaghcor castlebaldwin": "Sligo",
    "annaghieran shercock": "Cavan",
    "annaghkilly newbliss": "Monaghan",
    "annaghmore blue ball": "Offaly",
    "annaghmore gurteen": "Sligo",
    "annaghmore kinnitty birr": "Offaly",
    "annaghmore strokestown": "Roscommon",
    "annaghnamaddoo kilnagross carr": "Leitrim",
    "annaghselherny kilnagross carr": "Leitrim",
    "annagola glaslough": "Monaghan",
    "annahagh scotstown": "Monaghan",
    "annalecky dunlavin": "Wicklow",
    "annalough stranoden": "Monaghan",
    "annamoe": "Wicklow",
    "annamult stoneyford": "Kilkenny",
    "annaveagh scotshouse": "Monaghan",
    "anneville clonard": "Meath",
    "anneville hill of down": "Meath",
    "anney latton": "Monaghan",
    "annfield dunbell": "Kilkenny",
    "annies scotshouse": "Monaghan",
    "annmount glounthaune": "Cork",
    "annyalla": "Monaghan",
    "annyalty ballinode": "Monaghan",
    "anveyerg latton": "Monaghan",
    "arabella ballymacelligott": "Kerry",
    "aras mhic niallais rosses point": "Sligo",
    "archdeaconary kells": "Meath",
    "archerstown ashbourne": "Meath",
    "archerstown delvin": "Westmeath",
    "archerstown demesne ashbourne": "Meath",
    "archerstown durrow": "Laois",
    "ard na ri bruff": "Limerick",
    "ardabrone dromard": "Sligo",
    "ardacluggin eyeries": "Cork",
    "ardaloo jenkinstown": "Louth",
    "ardamine": "Wexford",
    "ardamore ballydavid": "Kerry",
    "ardane bansha": "Tipperary",
    "ardanragh lenamore": "Monaghan",
    "ardaprior castlecor": "Cork",
    "ardattin": "Carlow",
    "ardaturr church hill": "Donegal",
    "ardavagga shinrone birr": "Offaly",
    "ardboghil edgeworthstown": "Longford",
    "ardbohill rathkeale": "Limerick",
    "ardborra drumraney": "Westmeath",
    "ardbrack kinsale": "Cork",
    "ardbrackan kilmessan": "Meath",
    "ardcanaught castlemaine": "Kerry",
    "ardcanaught west castlemaine": "Kerry",
    "ardcath garristown": "Meath",
    "ardclough straffan": "Kildare",
    "ardcolm castlebridge": "Wexford",
    "ardcost cross portmagee": "Kerry",
    "ardcost west portmagee": "Kerry",
    "ardcotton collooney": "Sligo",
    "ardcrone farranfore": "Kerry",
    "ardellis athy": "Kildare",
    "ardellis lower athy": "Kildare",
    "ardellis upper athy": "Kildare",
    "ardenagh great foulksmills": "Wexford",
    "ardenagh great taghmon": "Wexford",
    "ardenew longwood": "Meath",
    "arderawinny schull": "Cork",
    "ardeskin donegal": "Donegal",
    "ardfrail oldcastle": "Meath",
    "ardginny killybrone": "Cavan",
    "ardgivna templeboy": "Sligo",
    "ardkeenagh boyle": "Roscommon",
    "ardkeeran riverstown": "Sligo",
    "ardkill kilmaine": "Mayo",
    "ardkilly sandycove kinsale": "Cork",
    "ardleag shanballymore": "Cork",
    "ardleny kilnaleck": "Cavan",
    "ardlow virginia": "Cavan",
    "ardmanagh schull": "Cork",
    "ardmone east canningstown": "Cavan",
    "ardmoneen glenfarne": "Leitrim",
    "ardmoyle boyle": "Roscommon",
    "ardnaglass lower grange": "Sligo",
    "ardnaglass pettigo": "Donegal",
    "ardnagross finea": "Cavan",
    "ardnagross kilmead athy": "Kildare",
    "ardnahinch shanagarry": "Cork",
    "ardnamoher galbally": "Limerick",
    "ardnatagle obriensbridge": "Clare",
    "ardneeskan drumfin": "Sligo",
    "ardnurcher horseleap": "Westmeath",
    "ardogeena durrus": "Cork",
    "ardra castlelyons": "Cork",
    "ardralla church cross skibbere": "Cork",
    "ardree athy": "Kildare",
    "ardrums little summerhill": "Meath",
    "ards fossa": "Kerry",
    "ards keenagh": "Longford",
    "ardsallagh beg": "Westmeath",
    "ardsallagh more": "Westmeath",
    "ardscull athy": "Kildare",
    "ardsoreen boyle": "Roscommon",
    "ardtrasna ballinfull": "Sligo",
    "ardtully beg riverstown": "Sligo",
    "arduns gweedore": "Donegal",
    "ardunsaghan corrawaleen": "Monaghan",
    "ardvarney glenfarne": "Leitrim",
    "arguga fivealley birr": "Offaly",
    "arnasbrack ballygawley": "Sligo",
    "arnestown new ross": "Wexford",
    "arodstown summerhill": "Meath",
    "arragh more carrig birr": "Offaly",
    "arthurstown": "Wexford",
    "artramon castlebridge": "Wexford",
    "asdee": "Kerry",
    "asdee east asdee": "Kerry",
    "ash hill kilmallock": "Limerick",
    "ashbourne": "Meath",
    "ashbourne industrial estate as": "Meath",
    "ashbrook strokestown": "Roscommon",
    "ashbury roscrea": "Tipperary",
    "ashewood ashbourne": "Meath",
    "ashfield clonard": "Meath",
    "ashford": "Wicklow",
    "ashgrove ballybrittas": "Laois",
    "ashgrove bansha": "Tipperary",
    "ashgrove kildangan monasterevi": "Kildare",
    "ashill ballymacelligott": "Kerry",
    "ashleam monkstown": "Cork",
    "askakeagh ballinglen": "Wicklow",
    "askamore": "Wexford",
    "askinamoe ferns": "Wexford",
    "assagart foulksmills": "Wexford",
    "assaly little killinick": "Wexford",
    "atateemore slieverue": "Kilkenny",
    "athasselabbey golden": "Tipperary",
    "athboy": "Meath",
    "athea": "Limerick",
    "athea lower athea": "Limerick",
    "athgaine kells": "Meath",
    "athronan dunsany": "Meath",
    "athy": "Kildare",
    "attymurtagh castledaly": "Westmeath",
    "augha nurney": "Kildare",
    "aughaboura athy": "Kildare",
    "aughaboy ferbane": "Offaly",
    "aughacasla south camp": "Kerry",
    "aughacasla south castlegregory": "Kerry",
    "aughafan castletown": "Wexford",
    "aughaleigue beg kilcrohane": "Cork",
    "aughamore kilmainhamwood": "Meath",
    "aughamore roosky": "Roscommon",
    "aughanursan milford": "Donegal",
    "augharan aghavas": "Leitrim",
    "augharrow tullaghan": "Leitrim",
    "aughavogue killybegs": "Donegal",
    "aughermon ballymitty": "Wexford",
    "aughermon taghmon": "Wexford",
    "augherskea drumree": "Meath",
    "augheygault big drumkeen": "Donegal",
    "aughfad taghmon": "Wexford",
    "aughills castlemaine": "Kerry",
    "aughnadrin laghey": "Donegal",
    "aughnagaddy ramelton": "Donegal",
    "aughnagomaun horse and jockey ": "Tipperary",
    "aughnahilla stradbally": "Laois",
    "aught muff": "Donegal",
    "aughterra kilnaleck": "Cavan",
    "auniskirtane rathmore": "Kerry",
    "avalreagh clontibret": "Monaghan",
    "ayle oola": "Limerick",
    "badgerhill kill": "Kildare",
    "baile na ngall beag an rinn du": "Waterford",
    "balcarrick donabate": "Dublin",
    "baldwinstown garristown": "Meath",
    "balgeeth ardcath garristown": "Meath",
    "balgree carnaross": "Meath",
    "balkinstown nurney": "Kildare",
    "balleek durrow": "Laois",
    "balleen hill freshford": "Kilkenny",
    "balleese lower rathdrum": "Wicklow",
    "balleven ballycallan": "Kilkenny",
    "ballincar rosses point": "Sligo",
    "ballincarrig west firies": "Kerry",
    "ballincarroona knocklong": "Limerick",
    "ballincash upper oulart": "Wexford",
    "ballinclare ballycanew": "Wexford",
    "ballinclare knockbridge": "Louth",
    "ballincloghan rahan": "Offaly",
    "ballincloher cross lixnaw": "Kerry",
    "ballincloher north lixnaw": "Kerry",
    "ballincloher west lixnaw": "Kerry",
    "ballincor shinrone birr": "Offaly",
    "ballincota ventry": "Kerry",
    "ballincraheen lixnaw": "Kerry",
    "ballincrossig causeway": "Kerry",
    "ballincrossig glanmire": "Cork",
    "ballincur kinnitty birr": "Offaly",
    "ballincur rahan": "Offaly",
    "ballincurrig leamlara": "Cork",
    "ballincurry curry": "Sligo",
    "ballindaggin": "Wexford",
    "ballindarra birr": "Offaly",
    "ballindeenisk belgooly": "Cork",
    "ballinderry four mile house": "Roscommon",
    "ballinderry moate": "Westmeath",
    "ballindinas barntown": "Wexford",
    "ballindoney ballywilliam": "Wexford",
    "ballindoon riverstown": "Sligo",
    "ballindrennan kilcormac": "Offaly",
    "ballindrum athy": "Kildare",
    "ballinduganig ballyseedy trale": "Kerry",
    "balline martinstown kilmallock": "Limerick",
    "ballineadig ovens": "Cork",
    "ballinealoe coole": "Westmeath",
    "ballinearla kilmacow": "Kilkenny",
    "ballinesker curracloe": "Wexford",
    "ballinfull": "Sligo",
    "ballingaddy kilmallock": "Limerick",
    "ballingarrane shanagarry": "Cork",
    "ballinglanna causeway": "Kerry",
    "ballinglanna dunquin": "Kerry",
    "ballinglanna glanmire": "Cork",
    "ballinglanna kilworth": "Cork",
    "ballinglen": "Wicklow",
    "ballingly wellingtonbridge": "Wexford",
    "ballingohig knockraha": "Cork",
    "ballingrane rathkeale": "Limerick",
    "ballinguile kiltegan": "Wicklow",
    "ballinguilla crookstown": "Cork",
    "ballinhalla clogheen": "Tipperary",
    "ballinillane ballyhar killarne": "Kerry",
    "balliniry ramsgrange": "Wexford",
    "ballinivarrig firies": "Kerry",
    "ballinkeel ballymurn": "Wexford",
    "ballinkillen muine bheag": "Carlow",
    "ballinknockane ballydavid": "Kerry",
    "ballinknockane camp": "Kerry",
    "ballinlig broadford": "Clare",
    "ballinlina golden": "Tipperary",
    "ballinloghig ballydavid": "Kerry",
    "ballinlug rathconrath": "Westmeath",
    "ballinlyna upper kilfinane": "Limerick",
    "ballinn delvin": "Westmeath",
    "ballinode": "Monaghan",
    "ballinookera whitegate": "Cork",
    "ballinora waterfall": "Cork",
    "ballinowlart north clonbullogu": "Offaly",
    "ballinphull cliffoney": "Sligo",
    "ballinphull templeboy": "Sligo",
    "ballinphunta cratloe": "Clare",
    "ballinra screen": "Wexford",
    "ballinrally camross": "Laois",
    "ballinree killyon birr": "Offaly",
    "ballinroan lower kiltegan": "Wicklow",
    "ballinroddy edgeworthstown": "Longford",
    "ballinrooaun screen": "Wexford",
    "ballinruddery listowel": "Kerry",
    "ballinskelligs": "Kerry",
    "ballintaggart callan": "Kilkenny",
    "ballintee dunnamaggin": "Kilkenny",
    "ballinteen inniscrone": "Sligo",
    "ballintemple cloughjordan": "Tipperary",
    "ballintemple dundrum": "Tipperary",
    "ballintemple dunquin": "Kerry",
    "ballintemple walsh island": "Offaly",
    "ballinteosig killeagh": "Cork",
    "ballintlea upper timahoe": "Laois",
    "ballintleave commons glenbeigh": "Kerry",
    "ballintleave glenbeigh": "Kerry",
    "ballintober hollywood": "Wicklow",
    "ballintober moydow": "Longford",
    "ballintober rosemount": "Westmeath",
    "ballintogher": "Sligo",
    "ballintogher ballybrittas": "Laois",
    "ballintogher killenaule": "Tipperary",
    "ballintogher lixnaw": "Kerry",
    "ballintourig currow": "Kerry",
    "ballinvalley castlewarren": "Kilkenny",
    "ballinvalley delvin": "Westmeath",
    "ballinvalley kildavin": "Carlow",
    "ballinvally coolaney": "Sligo",
    "ballinvally killeigh": "Offaly",
    "ballinvally oldcastle": "Meath",
    "ballinvinny glanmire": "Cork",
    "ballinvoher kilworth": "Cork",
    "ballinvoher lixnaw": "Kerry",
    "ballinvreena martinstown kilma": "Limerick",
    "ballinvriskig whites cross": "Cork",
    "ballinvuskig west mourneabbey": "Cork",
    "ballitore": "Kildare",
    "ballitore hill ballitore": "Kildare",
    "ballon": "Carlow",
    "ballon cross ballon": "Carlow",
    "ballon hill ballon": "Carlow",
    "balloughton wellingtonbridge": "Wexford",
    "ballower ballon": "Carlow",
    "ballure clogherevagh": "Sligo",
    "ballure culleens": "Sligo",
    "ballyadam kilmuckridge": "Wexford",
    "ballyadams ballitore": "Kildare",
    "ballyadding ballybrittas": "Laois",
    "ballyandrew doneraile": "Cork",
    "ballyandrew ferns": "Wexford",
    "ballyanne new ross": "Wexford",
    "ballyara tubbercurry": "Sligo",
    "ballyarkane castlemaine": "Kerry",
    "ballyarr ramelton": "Donegal",
    "ballyarrell killygordon": "Donegal",
    "ballyavill geashill": "Offaly",
    "ballybane liscarroll": "Cork",
    "ballybannon milford": "Donegal",
    "ballybarney narraghmore": "Kildare",
    "ballybawn rathnure": "Wexford",
    "ballybeg ballymacelligott": "Kerry",
    "ballybeg brosna birr": "Offaly",
    "ballybeg ferns": "Wexford",
    "ballybeg glenbrien": "Wexford",
    "ballybeg killinick": "Wexford",
    "ballybeg knocknahur": "Sligo",
    "ballybeg rathnew": "Wicklow",
    "ballybeg screen": "Wexford",
    "ballybeg upper ballynonty thur": "Tipperary",
    "ballybin ashbourne": "Meath",
    "ballybit big rathvilly": "Carlow",
    "ballyboes falcarragh": "Donegal",
    "ballyboggan barntown": "Wexford",
    "ballyboggan clonard": "Meath",
    "ballyboggan lower castlebridge": "Wexford",
    "ballybough tomhaggard": "Wexford",
    "ballyboughan": "Dublin",
    "ballybought athy": "Kildare",
    "ballybought ballymore eustace": "Kildare",
    "ballybought durrow": "Laois",
    "ballyboy east clogheen": "Tipperary",
    "ballyboy ferns": "Wexford",
    "ballyboy kilcormac": "Offaly",
    "ballyboy west clogheen": "Tipperary",
    "ballybrack ballydavid": "Kerry",
    "ballybrack carrignavar": "Cork",
    "ballybrack firies": "Kerry",
    "ballybrack moville": "Donegal",
    "ballybrack waterville": "Kerry",
    "ballybreen kiltealy": "Wexford",
    "ballybricken grange kilmallock": "Limerick",
    "ballybride": "Dublin",
    "ballybride upper conna": "Cork",
    "ballybrien ballymacarbry": "Tipperary",
    "ballybritt roscrea": "Tipperary",
    "ballybrittas": "Laois",
    "ballybrittas bree": "Wexford",
    "ballybrogan lecarrow": "Roscommon",
    "ballybrollaghan mountcharles": "Donegal",
    "ballybrophy": "Laois",
    "ballybrown streamstown": "Westmeath",
    "ballybruncullim ballycumber": "Offaly",
    "ballybuckley bree": "Wexford",
    "ballybunnion": "Kerry",
    "ballybur upper cuffesgrange": "Kilkenny",
    "ballyburly rhode": "Offaly",
    "ballycahane portlaw": "Waterford",
    "ballycahill templemore": "Tipperary",
    "ballycallan": "Kilkenny",
    "ballycanew": "Wexford",
    "ballycanon donadea": "Kildare",
    "ballycarney": "Wexford",
    "ballycarron golden": "Tipperary",
    "ballycarron jenkinstown": "Louth",
    "ballycatoo cloyne": "Cork",
    "ballyclamasy summerfield yough": "Cork",
    "ballyclare ferbane": "Offaly",
    "ballyclare lanesborough": "Longford",
    "ballyclare longwood": "Meath",
    "ballycleary castletown": "Wexford",
    "ballyclemock newbawn": "Wexford",
    "ballycloghan carrickboy": "Longford",
    "ballycloghduff moate": "Westmeath",
    "ballyclogher streamstown": "Westmeath",
    "ballyclough": "Cork",
    "ballyclough kilworth": "Cork",
    "ballyclovan callan": "Kilkenny",
    "ballycollin eglish birr": "Offaly",
    "ballycollin lower kilcormac": "Offaly",
    "ballycommane tournafulla": "Limerick",
    "ballycommon": "Tipperary",
    "ballycon daingean": "Offaly",
    "ballyconneely newmarket-on-fer": "Clare",
    "ballyconry lisselton": "Kerry",
    "ballycoolan stradbally": "Laois",
    "ballycoolid donaghmore": "Laois",
    "ballycommane tournafulla": "Limerick",
    "ballycommon": "Tipperary",
    "ballycon daingean": "Offaly",
    "ballyconneely newmarket-on-fer": "Clare",
    "ballyconry lisselton": "Kerry",
    "ballycoolan stradbally": "Laois",
    "ballycoolid donaghmore": "Laois",
    "ballycormack shanagolden": "Limerick",
    "ballycorney killaloe": "Clare",
    "ballycowan tagoat": "Wexford",
    "ballycreggan kiltoom": "Roscommon",
    "ballycrenane ballymacoda": "Cork",
    "ballycrenane beg mogeely": "Cork",
    "ballycrovane eyeries": "Cork",
    "ballycullane": "Kerry",
    "ballycullane glin": "Limerick",
    "ballycullane kilmallock": "Limerick",
    "ballycumisk schull": "Cork",
    "ballyda danesfort": "Kilkenny",
    "ballyda rathcormac": "Cork",
    "ballydarragh craanford": "Wexford",
    "ballydavid north": "Kerry",
    "ballyderown kilworth": "Cork",
    "ballydevitt beg donegal": "Donegal",
    "ballydevitt donegal": "Donegal",
    "ballydevlin goleen": "Cork",
    "ballydonnell thomastown": "Kilkenny",
    "ballydonnellan corrandulla": "Galway",
    "ballydonohoe lisselton": "Kerry",
    "ballydowel rathmoyle": "Tipperary",
    "ballydulea cobh": "Cork",
    "ballydwyer ballymacelligott": "Kerry",
    "ballyedmond": "Cork",
    "ballyeliane cobh": "Cork",
    "ballyfair curragh": "Kildare",
    "ballyfane carne": "Wexford",
    "ballyfarnoge screen": "Wexford",
    "ballyfarsoon monasterevin": "Kildare",
    "ballyfolan brittas": "Dublin",
    "ballygannon rathdrum": "Wicklow",
    "ballygawley": "Sligo",
    "ballygeale patrickswell": "Limerick",
    "ballygibba south kilmallock": "Limerick",
    "ballygillane big rosslare harb": "Wexford",
    "ballyglan dunmore east": "Waterford",
    "ballyglasheen kilfeacle": "Tipperary",
    "ballyglass four mile house": "Roscommon",
    "ballyglass lower": "Mayo",
    "ballyglass west loughglynn": "Roscommon",
    "ballyglass west loughlynn": "Roscommon",
    "ballyglissane rathcormac": "Cork",
    "ballyglunin": "Galway",
    "ballygoley riverstown": "Sligo",
    "ballygoman barntown": "Wexford",
    "ballygrennan listowel": "Kerry",
    "ballygriffin carrigeen waterfo": "Waterford",
    "ballygriffin golden": "Tipperary",
    "ballyhad lower rathdrum": "Wicklow",
    "ballyhahill west ballyhahill": "Limerick",
    "ballyhannan south quin": "Clare",
    "ballyharry lecamey": "Donegal",
    "ballyhea": "Cork",
    "ballyhean liscannor": "Clare",
    "ballyhenebery piltown": "Kilkenny",
    "ballyhickey quin": "Clare",
    "ballyhiernan kindrum": "Donegal",
    "ballyhillogue mourneabbey": "Cork",
    "ballyhip louisburgh": "Mayo",
    "ballyhogue": "Wexford",
    "ballyhomuck mullinavat": "Kilkenny",
    "ballyhone emly": "Tipperary",
    "ballyhooly": "Cork",
    "ballyhow beg screen": "Wexford",
    "ballyhussa kilmacthomas": "Waterford",
    "ballykeelaun parteen": "Clare",
    "ballykeeroge campile": "Wexford",
    "ballykenneally ballymacoda": "Cork",
    "ballykerrigan cloghan": "Offaly",
    "ballykillane hacketstown": "Carlow",
    "ballykilmurray": "Kerry",
    "ballykilmurry kilrossanty kilm": "Waterford",
    "ballykilty quin": "Clare",
    "ballykinsella tramore": "Waterford",
    "ballyknavin bridgetown": "Wexford",
    "ballylane east new ross": "Wexford",
    "ballylea dunlavin": "Wicklow",
    "ballyleary cobh": "Cork",
    "ballylibernagh bridgetown": "Wexford",
    "ballylickey": "Cork",
    "ballyliffin": "Donegal",
    "ballyloughan muine bheag": "Carlow",
    "ballyluddy pallasgreen": "Limerick",
    "ballylusky croom": "Limerick",
    "ballymacarbry upper ballymacar": "Waterford",
    "ballymacaw dunmore east": "Waterford",
    "ballymacdonnell broadford": "Clare",
    "ballymacredmond knockmore ball": "Mayo",
    "ballymacrown baltimore": "Cork",
    "ballymadrough donabate": "Dublin",
    "ballymagan buncrana": "Donegal",
    "ballymagaraghy lecamey": "Donegal",
    "ballymagroarty scotch ballintr": "Donegal",
    "ballymakea mullagh": "Clare",
    "ballymanus upper glenealy": "Wicklow",
    "ballymartin more": "Galway",
    "ballymena malin": "Donegal",
    "ballyminaun little killinick": "Wexford",
    "ballymoate glenealy": "Wicklow",
    "ballymoney": "Antrim",
    "ballymore": "Westmeath",
    "ballymore cobh": "Cork",
    "ballymore eustace": "Kildare",
    "ballymore ventry": "Kerry",
    "ballymore west ventry": "Kerry",
    "ballymorris bansha": "Tipperary",
    "ballymountain ferrybank waterf": "Waterford",
    "ballymoylan carrigatoher nenag": "Tipperary",
    "ballymurphy lower conna": "Cork",
    "ballymurray": "Mayo",
    "ballynabuck ballydavid": "Kerry",
    "ballynacarrig brittas bay": "Wicklow",
    "ballynagall knockdrin mullinga": "Westmeath",
    "ballynahallia moycullen": "Galway",
    "ballynahimmy clonaslee": "Laois",
    "ballynakill geashill": "Offaly",
    "ballynakill killashee": "Longford",
    "ballynakillew mountain laghey": "Donegal",
    "ballynalahessery north dungarv": "Waterford",
    "ballynamanagh east clarinbridg": "Galway",
    "ballynamona kells": "Meath",
    "ballynamona kilmuckridge": "Wexford",
    "ballynamona shanagarry": "Cork",
    "ballynaroon glounthaune": "Cork",
    "ballynelligan glebe lismore": "Waterford",
    "ballyness falcarragh": "Donegal",
    "ballynevin cullahill": "Laois",
    "ballynew cleggan": "Galway",
    "ballynew moyard": "Galway",
    "ballynoe cobh": "Cork",
    "ballynoe whites cross": "Cork",
    "ballynora waterfall": "Cork",
    "ballynoran charleville": "Cork",
    "ballyoonan omeath": "Louth",
    "ballyorley lower ferns": "Wexford",
    "ballyporeen": "Tipperary",
    "ballyquirk killeagh": "Cork",
    "ballyquirke killeagh": "Cork",
    "ballyquirke moycullen": "Galway",
    "ballyrogan upper redcross": "Wicklow",
    "ballyrue cross new ross": "Wexford",
    "ballysallagh johnswell": "Kilkenny",
    "ballysax curragh": "Kildare",
    "ballyshane doneraile": "Cork",
    "ballyshrule portumna": "Galway",
    "ballysimon monageer": "Wexford",
    "ballyskibbole ballymacoda": "Cork",
    "ballyslatteen golden": "Tipperary",
    "ballysteena cloughjordan": "Tipperary",
    "ballytarsna ballycullane": "Wexford",
    "ballytarsna muine bheag": "Carlow",
    "ballytarsna nurney": "Carlow",
    "ballythomas rathgormack": "Waterford",
    "ballytivnan": "Sligo",
    "ballytoohey termonbarry": "Roscommon",
    "ballytrasna lissarda": "Cork",
    "ballyvareen kildimo": "Limerick",
    "ballyvarra lisnagry": "Limerick",
    "ballyvary": "Mayo",
    "ballyvaskin south miltown malb": "Clare",
    "ballyvelogue patrickswell": "Limerick",
    "ballyvolan upper newcastle": "Wicklow",
    "ballywalter golden": "Tipperary",
    "ballyweeaun clonbur": "Galway",
    "ballywilliam": "Cork",
    "ballywilliam rathkeale": "Limerick",
    "ballywilliam shinrone birr": "Offaly",
    "ballywilliamroe muine bheag": "Carlow",
    "balrenny cross slane": "Meath",
    "balteenbrack ardfield": "Cork",
    "baltimore": "Cork",
    "baltray": "Louth",
    "banagher": "Offaly",
    "banemore listowel": "Kerry",
    "bannagh kanturk": "Cork",
    "bannow": "Wexford",
    "bannow moor bannow": "Wexford",
    "bannpark craanford": "Wexford",
    "bannus pettigo": "Donegal",
    "bansha": "Tipperary",
    "banteer": "Cork",
    "banteer west": "Cork",
    "barchuillia commons kilmacanog": "Wicklow",
    "barleyhill kingscourt": "Cavan",
    "barleyhill newmarket": "Cork",
    "barna": "Galway",
    "barna charleville": "Cork",
    "barna galbally": "Limerick",
    "barnahask curracloe": "Wexford",
    "barnane templemore": "Tipperary",
    "barnfield ballymacoda": "Cork",
    "barntown": "Wexford",
    "barraderry west kiltegan": "Wicklow",
    "barraghy loughmourne": "Louth",
    "barranastook old parish dungar": "Waterford",
    "barrettstown knocktopher": "Kilkenny",
    "barringtonsbridge lisnagry": "Limerick",
    "barroe": "Galway",
    "barroe south riverstown": "Sligo",
    "barry beg kiltoom": "Roscommon",
    "barry more kiltoom": "Roscommon",
    "bartragh inniscrone": "Mayo",
    "bauck saint mullins": "Carlow",
    "baunkyle corofin": "Clare",
    "baunoge callan": "Kilkenny",
    "baunoges north dunmore": "Waterford",
    "baurnahulla drimoleague": "Cork",
    "bauroe feakle": "Clare",
    "bawn mountain moydow": "Longford",
    "bawn the pigeons": "Kilkenny",
    "bawnea kilbrittain": "Cork",
    "bawnmore south kanturk": "Cork",
    "bawnogues straffan": "Kildare",
    "bawntaaffe monasterboice": "Louth",
    "bayhill ramelton": "Donegal",
    "baytown the ward": "Dublin",
    "bealadangan": "Galway",
    "bedford listowel": "Kerry",
    "begnagh killashee": "Longford",
    "behagloss roscrea": "Tipperary",
    "beheens west listowel": "Kerry",
    "belcruit burtonport": "Donegal",
    "belcruit kincasslagh": "Donegal",
    "belgooly": "Cork",
    "bellair moynalty": "Meath",
    "bellcotton termonfeckin": "Louth",
    "bellevue demesne delgany": "Wicklow",
    "bellville monivea": "Galway",
    "bellvue cobh": "Cork",
    "belmont cullahill": "Laois",
    "belrobin kilkerley": "Louth",
    "beltany raphoe": "Donegal",
    "beltichburne termonfeckin": "Louth",
    "belvelly cobh": "Cork",
    "bengour east newcestown": "Cork",
    "bethlehem the pigeons": "Kilkenny",
    "billary pettigo": "Donegal",
    "billeragh listowel": "Kerry",
    "binnion clonmany": "Donegal",
    "binroe bruckless": "Donegal",
    "birchill donegal": "Donegal",
    "birdhill": "Clare",
    "birdstown burnfoot": "Donegal",
    "birr": "Offaly",
    "birragh kingscourt": "Cavan",
    "bishopscourt ballygunner waterf": "Waterford",
    "bishopshall kilmacow": "Kilkenny",
    "bishopsland ballymore eustace": "Kildare",
    "bishopswood dundrum": "Tipperary",
    "blackbull banagher": "Offaly",
    "blackhill kill": "Kildare",
    "blacknock kilmeaden": "Waterford",
    "blackstairs hollyford": "Tipperary",
    "blackstone bridge lower killeens": "Cork",
    "blairstown castlefin": "Donegal",
    "blossomgrove glanmire": "Cork",
    "bocarnagh glengarriff": "Cork",
    "bogganfin ballymurray": "Mayo",
    "bohaun north killavally westpo": "Mayo",
    "boher ballycumber": "Offaly",
    "boherboy saggart": "Dublin",
    "bohergeela meanus kilmallock": "Limerick",
    "bohernabreena": "Dublin",
    "bohernagraga pallasgreen": "Limerick",
    "bohola": "Mayo",
    "ballyellane cobh": "Cork",
    "ballymoney": "Wexford",
    "ballywalter golden": "Tipperary",
    "bishopscourt ballygunner water": "Waterford",
    "boley little nurney": "Kildare",
    "boley shillelagh": "Wicklow",
    "boliska oughter spiddal": "Galway",
    "bonestown dunshaughlin": "Meath",
    "booladuff inagh": "Clare",
    "boolyglass hugginstown": "Kilkenny",
    "borelia glenbrien": "Wexford",
    "borrmount bree": "Wexford",
    "bottomstown hospital": "Limerick",
    "boulia firies": "Kerry",
    "boyher rockcorry": "Monaghan",
    "boyle": "Roscommon",
    "boyle business park deerpark b": "Roscommon",
    "braade kincasslagh": "Donegal",
    "brackaharagh caherdaniel": "Kerry",
    "brackary more manorhamilton": "Leitrim",
    "brackloon fuerty": "Roscommon",
    "bracklyn": "Westmeath",
    "breaffa west lissycasey": "Clare",
    "breaffy house resort breaghwy": "Mayo",
    "breandrum mohill": "Leitrim",
    "bree": "Wexford",
    "bree castleblayney": "Monaghan",
    "breenross mohill": "Leitrim",
    "breeogue knocknahur": "Sligo",
    "breska beg clarina": "Limerick",
    "brickhill east cratloe": "Clare",
    "bridgend": "Donegal",
    "bridgetown": "Wexford",
    "bridgetown laghey": "Donegal",
    "brinlack middle brinlack": "Donegal",
    "brittas bay": "Wicklow",
    "broadford": "Clare",
    "broadwood ballynagore": "Westmeath",
    "brockagh clonaslee": "Laois",
    "brockagh lower glenfarne": "Leitrim",
    "brockagh saint johnston": "Donegal",
    "brocklagh drumlish": "Longford",
    "brooklodge east glanmire": "Cork",
    "broomscourt charleville": "Cork",
    "brosna": "Kerry",
    "brownhill ovens": "Cork",
    "brownstown churchtown": "Meath",
    "brownstown freshford": "Kilkenny",
    "brownstown little curragh": "Kildare",
    "brownstown new ross": "Wexford",
    "brownstown upper curragh": "Kildare",
    "brownswood portlaw": "Waterford",
    "bruach na habhann spiddal": "Galway",
    "bruff": "Limerick",
    "buckill frenchpark": "Roscommon",
    "bulcaun charlestown": "Mayo",
    "bulgan glynn": "Wexford",
    "bunbeg": "Donegal",
    "buncrana": "Donegal",
    "bundoon killoe": "Longford",
    "bunn malin": "Donegal",
    "bunnahevna more corrandulla": "Galway",
    "burncourt": "Tipperary",
    "burnfoot": "Donegal",
    "burnfort": "Cork",
    "burnthouse rosscahill": "Galway",
    "burreen virginia": "Cavan",
    "burrow rosslare": "Wexford",
    "bushfield oranmore": "Galway",
    "butlerswood windgap": "Kilkenny",
    "cabra three mile house": "Monaghan",
    "cabragh tara": "Meath",
        "caher coolrain": "Laois",
    "caher kilcrohane": "Cork",
    "caheragh drimoleague": "Cork",
    "cahercastle craughwell": "Galway",
    "caherconlish": "Limerick",
    "caherconnell kilfenora": "Clare",
    "caherdaha kilnamartyra macroom": "Cork",
    "caherdaniel": "Kerry",
    "caherkereen kilnamartyra macro": "Cork",
    "caherline lisnagry": "Limerick",
    "cahermurphy kilmihil": "Clare",
    "cahernabrock shrule": "Mayo",
    "cahernagh ballyhahill": "Limerick",
    "caherogan miltown malbay": "Clare",
    "caherolickane durrus": "Cork",
    "cairn silverstream": "Monaghan",
    "caladh chulain lettermullen": "Galway",
    "caldragh knock": "Mayo",
    "calhame bruckless": "Donegal",
    "calhame castlefin": "Donegal",
    "callan": "Kilkenny",
    "callancor drumkeen": "Donegal",
    "callow roundstone": "Galway",
    "callowhill upper newtownmountkennedy": "Wicklow",
    "caltra culleens": "Sligo",
    "camata malin": "Donegal",
    "camblin new ross": "Wexford",
    "camlisk more edgeworthstown": "Longford",
    "camolin": "Wexford",
    "campile": "Wexford",
    "camross foulksmills": "Wexford",
    "cannafahy callan": "Kilkenny",
    "capnagoul freemount": "Cork",
    "cappagh beg kinvara": "Galway",
    "cappagh curraghroe": "Roscommon",
    "cappantymore meelick": "Clare",
    "capparattin donohill": "Tipperary",
    "capparoe greenane": "Tipperary",
    "cappoquin": "Waterford",
    "cappoquin demesne cappoquin": "Waterford",
    "cappyantanvally glencar": "Kerry",
    "caragh bridge glenbeigh": "Kerry",
    "carhoo kilcully": "Cork",
    "carhoo kildorrery": "Cork",
    "carhoo north riverstick": "Cork",
    "carhoo south riverstick": "Cork",
    "carker doneraile": "Cork",
    "carlanstown": "Meath",
    "carn smithborough": "Monaghan",
    "carnagh east kiltoom": "Roscommon",
    "carnamadra kinvara": "Galway",
    "carnane fedamore": "Limerick",
    "carnaross": "Meath",
    "carnaveagh latton": "Monaghan",
    "carnew": "Wicklow",
    "carney": "Sligo",
    "carnmore east oranmore": "Galway",
    "carnmore oranmore": "Galway",
    "carnowen raphoe": "Donegal",
    "carolina buncrana": "Donegal",
    "carraward dromcollogher": "Limerick",
    "carrick": "Donegal",
    "carrick west laghey": "Donegal",
    "carrickaderry clontibret": "Monaghan",
    "carrickaneena mountpleasant": "Louth",
    "carrickastuck hackballscross": "Louth",
    "carrickataskin derrybeg": "Donegal",
    "carrickatober crosskeys": "Cavan",
    "carrickbaggot grangebellew": "Louth",
    "carrickcarnan ravensdale": "Louth",
    "carrickedmond colehill": "Longford",
    "carrickfad fivemilebourne": "Leitrim",
    "carrickleck kingscourt": "Cavan",
    "carrickmacgarvey brinlack": "Donegal",
    "carrickmaquigley redcastle": "Donegal",
    "carrickmerlin hugginstown": "Kilkenny",
    "carrig east clarina": "Limerick",
    "carrigadrohid carrigadrohid": "Cork",
    "carrigagh moynalty": "Meath",
    "carrigaholt": "Clare",
    "carriganine macroom": "Cork",
    "carrigans": "Donegal",
    "carrigart": "Donegal",
    "carrigavantry south tramore": "Waterford",
    "carrigboy durrus": "Cork",
    "carrigdownane kildorrery": "Cork",
    "carrigeen cappoquin": "Waterford",
    "carrigeen croom": "Limerick",
    "carrigeen east conna": "Cork",
    "carrigeenagowna tubbercurry": "Sligo",
    "carrigeennageragh big lemybrien": "Waterford",
    "carrigleigh inchigeelagh": "Cork",
    "carrigmahon monkstown": "Cork",
    "carrigmannon glynn": "Wexford",
    "carrignafoy cobh": "Cork",
    "carrigrohane": "Cork",
        "carrowbane beg loughill": "Limerick",
    "carrowbarra island smithborough": "Monaghan",
    "carrowbunnaun strandhill": "Sligo",
    "carrowcannon falcarragh": "Donegal",
    "carrowcannon lower falcarragh": "Donegal",
    "carrowhubbock south inniscrone": "Sligo",
    "carrowkeel clonfanlough": "Offaly",
    "carrowkeel miltown malbay": "Clare",
    "carrowkeel quigleys point": "Donegal",
    "carrowkeel swinford": "Mayo",
    "carrowmoneash oranmore": "Galway",
    "carrowmore knock": "Mayo",
    "carrowmore north mullagh": "Clare",
    "carrowmore scarriff": "Clare",
    "carrowmore shrule": "Mayo",
    "carronnaffe moville": "Donegal",
    "carronnalassan four mile house": "Roscommon",
    "carronaseer dunmore": "Galway",
    "carronoona largydonnell": "Leitrim",
    "carrontober tubbercurry": "Sligo",
    "carrowphadeen lecarrow": "Roscommon",
    "carrowreagh dromard": "Sligo",
    "carrowskeheen lahardane ballina": "Mayo",
    "cartown pallaskenry": "Limerick",
    "cartron collooney": "Sligo",
    "cartron creggs": "Roscommon",
    "cartron drumlish": "Longford",
    "cartron fivemilebourne": "Leitrim",
    "cartron kilcolgan": "Galway",
    "cartrongibbagh rossinver": "Leitrim",
    "cartronkillerdoo cliffoney": "Sligo",
    "cashleen renvyle": "Galway",
    "castle leslie estate glaslough": "Monaghan",
    "castlebanny mullinavat": "Kilkenny",
    "castlebellingham": "Louth",
    "castlebridge": "Wexford",
    "castledooey manorcunningham": "Donegal",
    "castledurrow demesne durrow": "Laois",
    "castlefergus quin": "Clare",
    "castlefin": "Donegal",
    "castlegarden thomastown": "Kilkenny",
    "castleholding roscrea": "Tipperary",
    "castlejane wood glanmire": "Cork",
    "castlejane woods glanmire": "Cork",
    "castlekevin roundwood": "Wicklow",
    "castlekieran carnaross": "Meath",
    "castlelloyd oola": "Limerick",
    "castlelyons": "Cork",
    "castlemahon": "Limerick",
    "castlematrix rathkeale": "Limerick",
    "castlemehigan crookhaven": "Cork",
    "castlemore crookstown": "Cork",
    "castlepark clonakenny roscrea": "Tipperary",
    "castlepollard": "Westmeath",
    "castlerickard longwood": "Meath",
    "castletogher williamstown": "Galway",
    "castletown": "Laois",
    "castletown donaghmore": "Laois",
    "castletown geoghegan": "Westmeath",
    "castletownroche": "Cork",
    "castlewarden straffan": "Kildare",
    "castlewilder abbeyshrule": "Longford",
    "cattaganstown killenaule": "Tipperary",
    "cellar nobber": "Meath",
    "channonrock louth": "Louth",
    "chapel gardens kilmore quay": "Wexford",
    "chapelstown": "Carlow",
    "charlestown": "Mayo",
    "charlestown kilmacow": "Kilkenny",
    "charleville": "Cork",
    "cheekpoint": "Waterford",
    "cherriestown murrintown": "Wexford",
    "christianstown castlebellingham": "Louth",
    "church bay crosshaven": "Cork",
    "church brae fahan": "Donegal",
    "church hill": "Donegal",
    "church hill cullen": "Cork",
    "church hill meadows raheen limerick": "Limerick",
    "churchfield castlemagner": "Cork",
    "churchland tinahely": "Wicklow",
    "churchtown kilcar": "Donegal",
    "churchtown tomhaggard": "Wexford",
    "citywest": "Dublin",
    "claddanure west kenmare": "Kerry",
    "clahane liscannor": "Clare",
    "clar redcastle": "Donegal",
    "clarcam donegal": "Donegal",
    "clare strokestown": "Roscommon",
    "clarinwood tramore": "Waterford",
    "clarlougheske donegal": "Donegal",
    "clash carrigrohane": "Cork",
    "clash kanturk": "Cork",
    "clashacrow freshford": "Kilkenny",
    "clashadoo durrus": "Cork",
    "clashmaguire clondrohid macroom": "Cork",
    "clashmore": "Waterford",
    "cleagh clonmany": "Donegal",
    "cleariestown": "Wexford",
    "cleen knockvicar boyle": "Roscommon",
    "cliddaun farranfore": "Kerry",
    "clieveragh listowel": "Kerry",
    "cliffoney": "Sligo",
    "cloghernagh east islandeady": "Mayo",
    "cloghacloka patrickswell": "Limerick",
    "cloghan four roads": "Roscommon",
    "cloghanesheskeen castlegregory": "Kerry",
    "clogharevaun kiltullagh athenry": "Galway",
    "cloghbally upper mullagh": "Clare",
    "cloghbrack upper cloghbrack": "Galway",
    "clogheen": "Tipperary",
    "clogher barnesmore": "Donegal",
    "clogher goleen": "Cork",
    "clogher lixnaw": "Kerry",
    "clogher more elphin": "Roscommon",
    "clogher rathkenny": "Meath",
    "cloghereen upper muckross killarney": "Kerry",
    "cloghran swords": "Dublin",
        "clonadra lanesborough": "Longford",
    "clonakenny roscrea": "Tipperary",
    "clonamona lower craanford": "Wexford",
    "clonaslee": "Laois",
    "clonbeg buncrana": "Donegal",
    "clonboy bridgetown": "Clare",
    "clonbur": "Galway",
    "cloncannon mountmellick": "Laois",
    "cloncarhee clonlara": "Clare",
    "cloncoher geashill": "Offaly",
    "clondalee more hill of down": "Meath",
    "clondavaddog churchill": "Donegal",
    "clondaw ferns": "Wexford",
    "clondoogan summerhill": "Meath",
    "clonfanlough": "Offaly",
    "clongill wilkinstown": "Meath",
    "clonkeeffy oldcastle": "Meath",
    "clonkeenkerrill colemanstown": "Galway",
    "clonlara": "Clare",
    "clonmaine ladysbridge": "Cork",
    "clonmany": "Donegal",
    "clonmines wellingtonbridge": "Wexford",
    "clonmoher bodyke": "Clare",
    "clonmore templemore": "Tipperary",
    "clonnageeragh castlepollard": "Westmeath",
    "clonough inch": "Wexford",
    "clonoughter clonlara": "Clare",
    "clonsaula johnstown": "Kilkenny",
    "clonshavoy lisnagry": "Limerick",
    "clontaff union hall": "Cork",
    "clontinty glanworth": "Cork",
    "clontymullan lenamore": "Donegal",
    "clonymeath summerhill": "Meath",
    "cloon and commons castleconnell": "Limerick",
    "cloonakilleg mount talbot": "Roscommon",
    "cloonard cloondara": "Longford",
    "cloonbanaun shrule": "Mayo",
    "cloonboo corrandulla": "Galway",
    "cloonbrin abbeyshrule": "Longford",
    "clooncarreen bornacoola": "Leitrim",
    "clooncon west glenamaddy": "Galway",
    "clooncor strokestown": "Roscommon",
    "clooncunnig ardfield": "Cork",
    "cloondara": "Longford",
    "cloone templemore": "Tipperary",
    "clooneen dromore west": "Sligo",
    "cloonevit mooydow": "Longford",
    "clooney castletown": "Laois",
    "clooneyogan lahinch": "Clare",
    "cloonfinish swinford": "Mayo",
    "cloonfiugh killashee": "Longford",
    "clooniffe moycullen": "Galway",
    "cloonkeen williamstown": "Galway",
    "cloonlaheen middle mullagh": "Clare",
    "cloonlara south glenamaddy": "Galway",
    "cloonlee kylebrack": "Galway",
    "cloonlyon charlestown": "Mayo",
    "cloonmacart drumlish": "Longford",
    "cloonmore dunmore": "Galway",
    "cloonshanville frenchpark": "Roscommon",
    "cloonslanor strokestown": "Roscommon",
    "cloontagh clonmany": "Donegal",
    "cloontarriv ballymacelligott": "Kerry",
    "cloontumpher bornacoola": "Leitrim",
    "cloonturk dromod": "Leitrim",
    "cloonusker scarriff": "Clare",
    "cloughduv crookstown": "Cork",
    "cloughjordan": "Tipperary",
    "cloughkeating patrickswell": "Limerick",
    "cloughlea sixmilebridge": "Clare",
    "clounbrane moyvane": "Kerry",
    "clonoughter glin": "Limerick",
    "clounts cross rathmore": "Kerry",
    "clownings straffan": "Kildare",
    "cloyne": "Cork",
    "cloysparra grange": "Tipperary",
    "clunelly quigleys point": "Donegal",
    "clybenane roscrea": "Tipperary",
    "clyduff rhode": "Offaly",
    "coalpits slane": "Meath",
    "cobh": "Cork",
    "codrum macroom": "Cork",
    "coguish kilcar": "Donegal",
    "coill dubh": "Kildare",
    "coill fada longwood": "Meath",
    "cois na habhainn kilcrohane": "Cork",
    "coismeig mor furbo": "Galway",
    "coldwood craughwell": "Galway",
    "colliersland north dunboyne": "Meath",
    "collinstown": "Westmeath",
    "collooney": "Sligo",
    "commons dromiskin": "Louth",
    "conna": "Cork",
    "convent cross dundrum": "Tipperary",
    "cool west valentia island": "Kerry",
    "coola kildinan": "Cork",
    "coolaclarig listowel": "Kerry",
    "coolagh callan": "Kilkenny",
    "coolamadra donard": "Wicklow",
    "coolaney": "Sligo",
    "coolard listowel": "Kerry",
    "coolaruane lixnaw": "Kerry",
       "coolatanavally carrigrohane": "Cork",
    "coolbawn ferns": "Wexford",
    "coolbeggan east tallow": "Waterford",
    "coolboa clashmore": "Waterford",
    "coolbunnia cheekpoint": "Waterford",
    "coolcarrigan coill dubh": "Kildare",
    "coolcon hollymount": "Mayo",
    "coolcower macroom": "Cork",
    "coolcreen lauragh": "Kerry",
    "cooldorragha portumna": "Galway",
    "coolduve lissarda": "Cork",
    "coole monasterevin": "Kildare",
    "coole west whites cross": "Cork",
    "coolearagh coill dubh": "Kildare",
    "cooleen tullogher": "Kilkenny",
    "coolfinn portlaw": "Waterford",
    "coolfore monasterboice": "Louth",
    "coolisteige clonlara": "Clare",
    "coolkeeran killinick": "Wexford",
    "coolmakee crookstown": "Cork",
    "coolmanagh upper hacketstown": "Carlow",
    "coolmeen kildysart": "Clare",
    "coolnahane kanturk": "Cork",
    "coolnaharragill upper glenbeigh": "Kerry",
    "coolnahorna marshalstown": "Wexford",
    "coolnakisha leighlinbridge": "Carlow",
    "coolnaleen camolin": "Wexford",
    "coolnatullagh carron": "Clare",
    "coolrainey inistioge": "Kilkenny",
    "coolroe burncourt": "Tipperary",
    "coolroe portlaw": "Waterford",
    "coolroe upper glenbeigh": "Kerry",
    "coolshaghtena": "Galway",
    "cooltegin woodstown": "Waterford",
    "coolyduff inniscara": "Cork",
    "coolyhane macroom": "Cork",
    "coolyrahilly kinsale": "Cork",
    "coomatallin drinagh": "Cork",
    "cooneys field myrtleville": "Cork",
    "coonmore rearcross": "Tipperary",
    "coopers cross castlebellingham": "Louth",
    "cooradarrigan schull": "Cork",
    "cooragurteen schull": "Cork",
    "coose loughmourne": "Clare",
    "cootehall boyle": "Roscommon",
    "cor na ron middle inverin": "Galway",
    "corbally kanturk": "Cork",
    "corbally rosenallis": "Laois",
    "corboy edgeworthstown": "Longford",
    "corcam stranorlar": "Donegal",
    "corcamore clarina": "Limerick",
    "corcorans cross new ross": "Wexford",
    "corderry knockbridge": "Louth",
    "cordoogan monasterboice": "Louth",
    "corduff coill dubh": "Kildare",
    "corduff gorvagh": "Leitrim",
    "corduff lusk": "Dublin",
    "cormackstown holycross": "Tipperary",
    "cormaclew walderstown": "Westmeath",
    "cormeen drum": "Monaghan",
    "cormey kingscourt": "Cavan",
    "cormey kingscourt kells": "Cavan",
    "cornakill kingscourt": "Cavan",
    "cornakill mullagh": "Cavan",
    "cornaman glenfarne": "Leitrim",
    "corofin": "Clare",
    "corr hill ballynahown": "Offaly",
    "corracloon flagmount": "Leitrim",
    "corracramp south dromod": "Leitrim",
    "corraduff rathcabin": "Tipperary",
    "corrahara shercock": "Cavan",
    "corralanna lismacaffrey": "Westmeath",
    "corranewy aghabog": "Monaghan",
    "corratober kingscourt": "Cavan",
    "correaly lismacaffrey": "Westmeath",
    "corriga": "Leitrim",
    "corrinshigo braddox": "Monaghan",
    "corryard drumshanbo": "Leitrim",
    "corrycholman kingscourt": "Cavan",
    "corsillagh newtownmountkennedy": "Wicklow",
    "corskeagh strokestown": "Roscommon",
    "corstown drumconrath": "Meath",
    "corthna schull": "Cork",
    "cortial kilkerley": "Louth",
    "corvin buncrana": "Donegal",
    "cottage hill commercial park limerick": "Limerick",
    "coulagh eyeries": "Cork",
    "courtmacsherry": "Cork",
    "cousinstown tomhaggard": "Wexford",
    "craan upper craanford": "Wexford",
    "craan wells": "Wexford",
    "craig muff": "Donegal",
    "craughwell": "Galway",
    "creeslough": "Donegal",
    "creeves donamon": "Roscommon",
    "creeves latton": "Monaghan",
    "creeves mount temple": "Westmeath",
    "cregg lahinch": "Clare",
    "creggane rosscarbery": "Cork",
    "creggaun glenamaddy": "Galway",
    "creggaun oranmore": "Galway",
    "crinnage craughwell": "Galway",
    "croagh": "Limerick",
    "croaghane foynes": "Limerick",
    "croanruss myshall": "Carlow",
    "crobally upper tramore": "Waterford",
    "crocknaraw moyard": "Galway",
    "croghan rhode": "Offaly",
    "crolly": "Donegal",
    "cromoge mountrath": "Laois",
    "crookhaven": "Cork",
    "croom": "Limerick",
    "cross of the wood tarbert": "Kerry",
    "crossaghy clontibret": "Monaghan",
    "crossalare laragh": "Monaghan",
    "crosshaven": "Cork",
    "crossmolina": "Mayo",
    "crossna knockvicar boyle": "Roscommon",
    "crover broomfield": "Monaghan",
    "crover mountnugent": "Cavan",
    "crows gap knockcroghery": "Roscommon",
    "cruit lower kincasslagh": "Donegal",
    "crumpane eyeries": "Cork",
    "cuillaun hollymount": "Mayo",
    "cuilmore gurteen": "Sligo",
    "culfore mountpleasant": "Louth",
    "cullagh beg drumcliffe": "Sligo",
    "culleen rathcabbin": "Tipperary",
    "culleenduff strandhill": "Sligo",
    "cullen riverstick": "Cork",
    "cullenagh kilmeaden": "Waterford",
    "cullenwaine cloughjordan": "Tipperary",
    "culliagh upper scramoge": "Roscommon",
    "cullies shercock": "Cavan",
    "cullyfad": "Longford",
    "cullyvore edgeworthstown": "Longford",
    "culmullin drumree": "Meath",
    "cummeen": "Sligo",

}




In [91]:
## Using Hospital; Attended to map to region if there is no hospital then map the pickuptown to county and then the region 

# HSE Regions mapping
hse_regions = {
    "HSE Dublin and North East": ["Cavan", "Monaghan", "Louth", "Meath", "Dublin"],
    "HSE Dublin and Midlands": ["Kildare", "Wicklow", "Laois", "Offaly", "Longford", "Westmeath", "Dublin"],
    "HSE Dublin and South East": ["Carlow", "Kilkenny", "Tipperary", "Waterford", "Wexford", "Wicklow", "Dublin"],
    "HSE Mid West": ["Clare", "Limerick", "Tipperary"],
    "HSE South West": ["Cork", "Kerry"],
    "HSE West and North West": ["Donegal", "Sligo", "Leitrim", "Mayo", "Galway", "Roscommon"]
}

# Hospital to region mapping
# Hospital to region mapping
hospital_to_region = {
    # HSE Dublin and North East - RCSI Hospitals
    "Beaumont Hospital": "HSE Dublin and North East",
    "Connolly Hospital": "HSE Dublin and North East",
    "CONNOLLY MEMORIAL HOSPITAL": "HSE Dublin and North East",
    "Our Lady of Lourdes Hospital, Drogheda": "HSE Dublin and North East",
    "BEAUMONT HOSPITAL": "HSE Dublin and North East",
    "CONNOLLY HOSPITAL": "HSE Dublin and North East",
    "LOURDES HOSPITAL DROGHEDA": "HSE Dublin and North East",
    "OUR LADYS NAVAN": "HSE Dublin and North East",
    "CAVAN GENERAL HOSPITAL": "HSE Dublin and North East",
    
    # HSE Dublin and Midlands - Dublin Midlands Hospital Group
    "Midland Regional Hospital, Tullamore": "HSE Dublin and Midlands",
    "St James's Hospital": "HSE Dublin and Midlands",
    "Tallaght University Hospital": "HSE Dublin and Midlands",
    "TULLAMORE HOSPITAL": "HSE Dublin and Midlands",
    "TULLAMORE GENERAL HOSPITAL": "HSE Dublin and Midlands",
    "ST JAMES HOSPITAL": "HSE Dublin and Midlands",
    "TALLAGHT HOSPITAL": "HSE Dublin and Midlands",
    "NAAS GENERAL HOSPITAL": "HSE Dublin and Midlands",
    "MULLINGAR REGIONAL HOSPITAL": "HSE Dublin and Midlands",
    "PORTLAOISE GENERAL HOSPITAL": "HSE Dublin and Midlands",
    "TEMPLE ST CHILDRENS HOSPITAL": "HSE Dublin and Midlands",
    "HERMITAGE MEDICAL CENTRE DUBLIN": "HSE Dublin and Midlands",
    
    # HSE Dublin and South East - Ireland East Hospital Group
    "Mater Misericordiae University Hospital": "HSE Dublin and South East",
    "St Vincent's University Hospital": "HSE Dublin and South East",
    "MATER HOSPITAL": "HSE Dublin and South East",
    "ST VINCENTS HOSPITAL": "HSE Dublin and South East",
    "WEXFORD GENERAL HOSPITAL": "HSE Dublin and South East",
    "ST LUKES HOSPITAL KILKENNY": "HSE Dublin and South East",
    "ST MICHAELS DUN LAOGHAIRE": "HSE Dublin and South East",
    "ST COLUMCILLES HOSPITAL LOUGHLINSTOWN": "HSE Dublin and South East",
    
    # HSE Mid West - UL Hospital Group
    "University Hospital Limerick": "HSE Mid West",
    "UNIVERSITY HOSPITAL LIMERICK": "HSE Mid West",
    "LIMERICK REGIONAL HOSPITAL": "HSE Mid West",
    "ENNIS GENERAL HOSPITAL": "HSE Mid West",
    "NENAGH GENERAL HOSPITAL": "HSE Mid West",
    "LIMERICK MATERNITY HOSPITAL": "HSE Mid West",
    "ST JOHNS HOSPITAL LIMERICK": "HSE Mid West",
    "PCI CENTRE LIMERICK": "HSE Mid West",
    
    # HSE South West - South/South West Hospital Group
    "Cork University Hospital": "HSE South West",
    "University Hospital Kerry": "HSE South West",
    "University Hospital Waterford": "HSE South West",
    "CORK UNIVERSITY HOSPITAL": "HSE South West",
    "KERRY GENERAL HOSPITAL": "HSE South West",
    "KERRY UNIVERSITY HOSPITAL": "HSE South West",
    "WATERFORD UNIVERSITY HOSPITAL": "HSE South West",
    "SOUTH TIPPERARY GENERAL": "HSE South West",
    "SOUTH TIPPERARY GENERAL HOSPITAL": "HSE South West",
    "MERCY UNIVERSITY HOSPITAL": "HSE South West",
    "BANTRY GENERAL HOSPITAL": "HSE South West",
    "MALLOW GENERAL HOSPITAL": "HSE South West",
    "CORK UNIVERSITY MATERNITY HOSP": "HSE South West",
    "BONS SECOURS HOSPITAL TRALEE": "HSE South West",
    
    # HSE West and North West - SAOLTA University Hospital Group
    "Letterkenny University Hospital": "HSE West and North West",
    "Mayo University Hospital": "HSE West and North West",
    "Sligo University Hospital": "HSE West and North West",
    "University Hospital Galway": "HSE West and North West",
    "LETTERKENNY GENERAL HOSPITAL": "HSE West and North West",
    "MAYO GENERAL HOSPITAL": "HSE West and North West",
    "SLIGO UNIVERSITY HOSPITAL": "HSE West and North West",
    "SLIGO GENERAL HOSPITAL": "HSE West and North West",
    "GALWAY UNIVERSITY HOSPITAL": "HSE West and North West",
    "PORTIUNCULA GENERAL HOSPITAL": "HSE West and North West",
    "ROSCOMMON GENERAL HOSPITAL": "HSE West and North West",
    "DONEGAL DISTRICT HOSPITAL": "HSE West and North West",
    "ST JOHNS HOSPITAL SLIGO": "HSE West and North West",
    
    # Northern Ireland hospitals (not part of HSE)
    "ALTNAGELVIN HOSPITAL": "Northern Ireland",
    "DAISY HILL HOSPTIAL NEWRY": "Northern Ireland",
    "CRAIGAVON AREA HOSPITAL": "Northern Ireland",
    "SOUTH WEST ACCUTE HOSPITAL": "Northern Ireland"
}

# Make all keys lowercase for case-insensitive lookup
hospital_to_region_lowercase = {k.lower(): v for k, v in hospital_to_region.items()}

# Get HSE region from county
def get_hse_region(county):
    if county is None:
        return None
        
    # Make county uppercase for case-insensitive comparison
    county_upper = county.upper()
    for region, counties in hse_regions.items():
        if any(county.upper() == c.upper() for c in counties):
            return region
    return None

# Function to determine region based on hospital and town
def determine_region(row):
    # First check if Hospital Attended has a value and is in our mapping
    if pd.notna(row["Hospital Attended"]) and row["Hospital Attended"] != "Unknown":
        # Try to match the hospital name (case-insensitive)
        hospital_lower = row["Hospital Attended"].lower()
        if hospital_lower in hospital_to_region_lowercase:
            return hospital_to_region_lowercase[hospital_lower]
            
        # If no direct match, check if any hospital name contains this string
        for hospital, region in hospital_to_region_lowercase.items():
            if hospital in hospital_lower or hospital_lower in hospital:
                return region
    
    # If we couldn't determine the region from the hospital, try using PickupTown
    if pd.notna(row["PickupTown"]):
        town = row["PickupTown"]
        
        # Try to find the county for this town (case-insensitive)
        county = None
        town_lower = town.lower()
        
        for t, c in town_to_county.items():
            if t.lower() == town_lower or t.lower() in town_lower:
                county = c
                break
        
        # If we found a county, get the HSE region
        if county:
            region = get_hse_region(county)
            if region:
                return region
    
    # If all else fails
    return "Unknown"

# Apply function to determine region for each row
nas_df["region"] = nas_df.apply(determine_region, axis=1)

# Optional: Display statistics on how many rows were assigned to each region
region_counts = nas_df["region"].value_counts()
print("Region assignment counts:")
print(region_counts)
print(f"Unknown regions: {region_counts.get('Unknown', 0)} out of {len(nas_df)} rows ({region_counts.get('Unknown', 0)/len(nas_df)*100:.2f}%)")


Region assignment counts:
region
HSE South West               37304
HSE West and North West      31774
HSE Dublin and North East    24293
HSE Dublin and South East    23615
HSE Dublin and Midlands      21067
HSE Mid West                 12887
Unknown                       4210
Northern Ireland                56
Name: count, dtype: int64
Unknown regions: 4210 out of 155206 rows (2.71%)


In [92]:
# Filter for rows with Unknown region
unknown_regions = nas_df[nas_df['region'] == 'Unknown']

# Check how many have a value in PickupTown
pickup_counts = unknown_regions['PickupTown'].notna().sum()
print(f"Unknown regions with PickupTown value: {pickup_counts} out of {len(unknown_regions)} ({pickup_counts/len(unknown_regions)*100:.2f}%)")

# Get the unique PickupTown values for unknown regions
unique_towns = unknown_regions['PickupTown'].dropna().unique()
print(f"Number of unique towns in unknown regions: {len(unique_towns)}")

# Display the most common towns in unknown regions
town_value_counts = unknown_regions['PickupTown'].value_counts().head(20)
print("\nTop 20 most common towns in unknown regions:")
print(town_value_counts)

print("\nAll towns not being matched:")
all_unique_towns = sorted(unknown_regions['PickupTown'].dropna().unique())
for town in all_unique_towns:
    print(f"- '{town}'")

Unknown regions with PickupTown value: 4209 out of 4210 (99.98%)
Number of unique towns in unknown regions: 1583

Top 20 most common towns in unknown regions:
PickupTown
STILLORGAN      162
KILLINEY        124
TRAMORE          62
NEW ROSS         55
KENMARE          48
KELLS            43
SALLYNOGGIN      43
SWORDS           42
KINSALE          39
LISTOWEL         36
KANTURK          35
MONASTEREVIN     34
PORTLAW          29
VIRGINIA         25
RIVERCHAPEL      24
ROSCREA          24
GLASTHULE        24
DELGANY          23
TEMPLEMORE       23
DONEGAL          22
Name: count, dtype: int64

All towns not being matched:
- 'BALLYWALTER GOLDEN'
- 'CARRIGEENNAGERAGH BIG LEMYBRIE'
- 'CARROWBARRA ISLAND SMITHBOROUG'
- 'CARROWNAFFE MOVILLE'
- 'CARROWNALASSAN FOUR MILE HOUSE'
- 'CARROWNASEER DUNMORE'
- 'CARROWNOONA LARGYDONNELL'
- 'CARROWNTOBER TUBBERCURRY'
- 'CARROWSKEHEEN LAHARDANE BALLIN'
- 'CLASHMAGUIRE CLONDROHID MACROO'
- 'CLOGGERNAGH EAST ISLANDEADY CA'
- 'CLOGHAREVAUN KILTULLAGH ATHENR'

In [93]:
# Filter for rows with Unknown region
unknown_regions = nas_df[nas_df['region'] == 'Unknown']

# Check count of Unknown regions
print(f"Total number of unknown regions: {len(unknown_regions)}")

# Count the most common Hospital Attended values for unknown regions
hospital_counts = unknown_regions['Hospital Attended'].value_counts().head(15)
print("\nMost common Hospital Attended values for unknown regions:")
print(hospital_counts)

Total number of unknown regions: 4210

Most common Hospital Attended values for unknown regions:
Hospital Attended
Unknown                      4208
GALWAY HOSPICE FOUNDATION       1
MONAGHAN GENERAL HOSPITAL       1
Name: count, dtype: int64


In [81]:
nas_df.head(10)

,date,Time of Day,PickupTown,DespatchCode,DespCodeDescription,Hospital Attended,region
0,2016-01-01,Night,DUNGLOE,17B01P,Falls - Possibly Dangerous body Area - Public ...,LETTERKENNY GENERAL HOSPITAL,HSE West and North West
1,2016-01-01,Night,BEAUMONT DUBLIN 9,17B01G,Falls - Possibly Dangerous body Area - On the ...,BEAUMONT HOSPITAL,HSE Dublin and North East
2,2016-01-01,Night,DROGHEDA,17B01,Falls - Possibly Dangerous body Area -,LOURDES HOSPITAL DROGHEDA,HSE Dublin and North East
3,2016-01-01,Night,BORRISOKANE,17B01P,Falls - Possibly Dangerous body Area - Public ...,PORTIUNCULA GENERAL HOSPITAL,HSE West and North West
4,2016-01-01,Night,WEXFORD,17B01G,Falls - Possibly Dangerous body Area - On the ...,Unknown,HSE Dublin and South East
5,2016-01-01,Night,KILLARNEY,17D04G,Falls - Not Alert - On the ground or floor,KERRY UNIVERSITY HOSPITAL,HSE South West
6,2016-01-01,Night,RATHNEENY WEST LAGHEY,17B01,Falls - Possibly Dangerous body Area -,SLIGO GENERAL HOSPITAL,HSE West and North West
7,2016-01-01,Morning,GLENAGEARY,17B00G,Falls - Bravo Override - On the ground or floor,ST VINCENTS HOSPITAL,HSE Dublin and South East
8,2016-01-01,Morning,DUNDALK,17B01,Falls - Possibly Dangerous body Area -,LOURDES HOSPITAL DROGHEDA,HSE Dublin and North East
9,2016-01-01,Morning,CAUSEWAY,17B01G,Falls - Possibly Dangerous body Area - On the ...,KERRY UNIVERSITY HOSPITAL,HSE South West


### Step 4.4: Check for any furthe rmissing values and drop unneeded columns

In [29]:
##check for NaN/null values
(nas_df.isna().sum()/nas_df.shape[0] * 100).sort_values()

date                   0.000000
Time of Day            0.000000
DespatchCode           0.000000
DespCodeDescription    0.000000
region                 0.000000
Hospital Attended      0.000000
PickupTown             0.008376
dtype: float64

In [19]:
nas_df.drop(columns=["EIRCODE ID"], inplace=True)

In [20]:
nas_df.drop(columns=["Randomised ID"], inplace=True)

In [21]:
nas_df.dropna(subset=['PickupTown'])

,DateCallForPerf,Time of Day,PickupTown,DespatchCode,DespCodeDescription,Hospital Attended,region
0,2016-01-01,Night,DUNGLOE,17B01P,Falls - Possibly Dangerous body Area - Public ...,LETTERKENNY GENERAL HOSPITAL,HSE West and North West
1,2016-01-01,Night,BEAUMONT DUBLIN 9,17B01G,Falls - Possibly Dangerous body Area - On the ...,BEAUMONT HOSPITAL,HSE Dublin and North East
2,2016-01-01,Night,DROGHEDA,17B01,Falls - Possibly Dangerous body Area -,LOURDES HOSPITAL DROGHEDA,HSE Dublin and North East
3,2016-01-01,Night,BORRISOKANE,17B01P,Falls - Possibly Dangerous body Area - Public ...,PORTIUNCULA GENERAL HOSPITAL,HSE West and North West
4,2016-01-01,Night,WEXFORD,17B01G,Falls - Possibly Dangerous body Area - On the ...,Unknown,Unknown
...,...,...,...,...,...,...,...
155201,2023-12-31,Evening,GLASSERCHOO GORTAHORK,30D03,Traumatic Injuries - Not Alert with Traumatic ...,LETTERKENNY GENERAL HOSPITAL,HSE West and North West
155202,2023-12-31,Evening,GORMANSTON,30B01,Traumatic Injuries - Possibly Dangerous Body Area,Unknown,Unknown
155203,2023-12-31,Evening,CLONMEL,17D04P,"Falls - Not Alert - Public place (street, carp...",Unknown,Unknown
155204,2023-12-31,Evening,DUN LAOGHAIRE,17B01,Falls - Possibly Dangerous body Area -,Unknown,Unknown


In [25]:
# Check some problematic examples
unknown_regions = nas_df[nas_df['region'] == 'Unknown']
print(f"Towns without regions: {unknown_regions['PickupTown'].unique()[:10]}")

Towns without regions: ['WEXFORD' 'CARROWCUBIC BALLYCASTLE' 'SHANKILL DUBLIN 18' 'CELBRIDGE'
 'NAVAN' 'CORK' 'SHRULE' 'SLIGO' 'CLONES' 'CABINTEELY DUBLIN 18']


In [22]:
nas_df = nas_df.rename(columns={"DateCallForPerf": "date"})

In [23]:
# Check for rows where 'hospital attended' is 'unknown'
unknown_rows = nas_df[nas_df['Hospital Attended'] == 'unknown']

# Display these rows
print(unknown_rows)

Empty DataFrame
Columns: [date, Time of Day, PickupTown, DespatchCode, DespCodeDescription, Hospital Attended, region]
Index: []


### Step 5: Save file to casv

In [24]:
nas_df.to_csv('/home/paulharford/college/project/project_data/nas/WEATHERED_nas_clean.csv', index=False)